# Epithelium

### graph / dataframe interface


## I/O

- hdf_snapshot


## topology

### graphs

- full graph
- cells graph
- junctions graph

### dataframes

- vertex df
- edge df
- triangles

### actions

- add vertex
- add cell
- add junction
- remove cell
- remove junction




## geometry

- position
- height / area / volume


## dynamics

- energy
- gradient
- optimisation
  + local
  + global


## filters

- cell vertices
- junction edges
- local
- active

## events

- division
- T1
- T3
- apoptosis


In [1]:
%pdb

Automatic pdb calling has been turned ON
Automatic pdb calling has been turned ON


In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import pandas as pd
import graph_tool.all as gt

from leg_joint.epithelium import Epithelium



In [3]:
eptm = Epithelium()

2015-03-16 11:15:08,075 -leg_joint.epithelium.epithelium -__init__ -- INFO -Instanciating epithelium 0
2015-03-16 11:15:08,076 -leg_joint.epithelium.epithelium -__init__ -- INFO -Created new graph
2015-03-16 11:15:08,228 -leg_joint.epithelium.epithelium -__init__ -- INFO -Initial cells
2015-03-16 11:15:08,228 -leg_joint.epithelium.epithelium -__init__ -- INFO -Initial junctions
2015-03-16 11:15:08,273 -leg_joint.epithelium.epithelium -__init__ -- INFO -Isotropic relaxation
/home/guillaume/anaconda/envs/python3/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
2015-03-16 11:15:08,487 -leg_joint.epithelium.epithelium -__init__ -- INFO -Periodic boundary
2015-03-16 11:15:08,490 -leg_joint.epithelium.epithelium -__init__ -- INFO -Update geometry
